In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import math
import joblib
import traceback

# Load model and feature list with comprehensive error handling
try:
    model = joblib.load('rf_model.joblib')
    feature_cols = joblib.load('rf_features.joblib')
    print("✅ Model files loaded successfully")
    print(f"Model type: {type(model)}")
    print(f"Feature columns count: {len(feature_cols)}")
except FileNotFoundError as e:
    print(f"❌ Model files not found: {e}")
    print("Please ensure 'rf_model.joblib' and 'rf_features.joblib' are in the current directory")
    raise
except Exception as e:
    print(f"❌ Error loading model files: {e}")
    raise

# Updated renovation costs with realistic pricing (standard costs)
RENOVATION_COSTS = {
    'wall_insulation': {
        'internal': 65,     # £55-75/m²
        'external': 120,    # £100-140/m²
        'cavity_fill': 22   # £18-26/m² - Cavity wall insulation fill
    },
    'roof_insulation': {
        'loft': 18,         # £15-22/m²
        'flat_roof': 80     # £70-90/m²
    },
    'glazing': {
        'double_glazing': 320,      # £280-360/m²
        'triple_glazing': 450,      # £400-500/m²
        'secondary_glazing': 150    # £130-170/m²
    },
    'heating_system': {
        # These costs are AFTER BUS grant (already deducted)
        'air_source_heat_pump': 6500,     # £14,000 - £7,500 BUS grant
        'ground_source_heat_pump': 12000, # £18,000 - £6,000 BUS grant
        'gas_boiler_upgrade': 2400,       # Standard cost
        'electric_boiler': 1800           # Standard cost
    }
}

# Energy costs from Ofgem Price Cap (October 2024)
ENERGY_COSTS = {
    'electricity': 0.285,  # £/kWh 
    'gas': 0.073,         # £/kWh
    'mixed': 0.18         # Average for mixed systems
}

# Government grants information
GOVERNMENT_GRANTS = {
    'BUS': {
        'air_source_heat_pump': 7500,
        'ground_source_heat_pump': 6000,
        'description': 'Boiler Upgrade Scheme (BUS) grants already deducted from heat pump costs above'
    },
    'ECO4': {
        'max_amount': 10000,
        'description': 'Energy Company Obligation - for eligible low-income households'
    }
}

def safe_float_conversion(value, default=0.0):
    """Safely convert value to float"""
    try:
        if value is None:
            return default
        return float(value)
    except (ValueError, TypeError):
        print(f"Warning: Could not convert {value} to float, using default {default}")
        return default

def safe_int_conversion(value, default=1):
    """Safely convert value to int"""
    try:
        if value is None:
            return default
        return int(float(value))  # Convert through float first to handle "2.0" strings
    except (ValueError, TypeError):
        print(f"Warning: Could not convert {value} to int, using default {default}")
        return default

def get_energy_cost_per_kwh(fuel_type):
    """Get energy cost per kWh with comprehensive error handling"""
    try:
        if fuel_type is None:
            return ENERGY_COSTS['mixed']
        
        fuel_type_str = str(fuel_type).lower().strip()
        
        if 'electric' in fuel_type_str:
            return ENERGY_COSTS['electricity']
        elif 'gas' in fuel_type_str:
            return ENERGY_COSTS['gas']
        else:
            return ENERGY_COSTS['mixed']
    except Exception as e:
        print(f"Warning: Error getting energy cost for {fuel_type}: {e}")
        return ENERGY_COSTS['mixed']

def calculate_actual_areas(total_floor_area, building_type='flat'):
    """Calculate actual renovation areas with comprehensive validation"""
    try:
        # Input validation and conversion
        floor_area = safe_float_conversion(total_floor_area, 60.0)
        
        if floor_area <= 0:
            print(f"Warning: Invalid floor area {floor_area}, using default 60")
            floor_area = 60.0
        
        if floor_area > 2000:  # Sanity check
            print(f"Warning: Very large floor area: {floor_area} m²")

        if building_type == 'flat':
            # Conservative calculation for flats
            wall_perimeter = math.sqrt(floor_area) * 2.5
            wall_area = wall_perimeter * 2.5 * 0.4  # Only 40% is external wall
            glazing_area = wall_area * 0.2  # 20% of wall area
            roof_area = floor_area
        else:  # house
            wall_area = floor_area * 1.8
            glazing_area = floor_area * 0.15
            roof_area = floor_area / math.cos(math.radians(30))
        
        # Ensure all areas are positive
        wall_area = max(wall_area, 1.0)
        glazing_area = max(glazing_area, 1.0)
        roof_area = max(roof_area, 1.0)
        
        return wall_area, glazing_area, roof_area
    
    except Exception as e:
        print(f"Error in calculate_actual_areas: {e}")
        # Return safe default values
        default_area = safe_float_conversion(total_floor_area, 60.0)
        return default_area * 0.3, default_area * 0.06, default_area

def determine_roof_type_from_location(floor_location, roof_type):
    """Determine final roof type based on floor location"""
    try:
        if floor_location == 'top floor':
            return str(roof_type) if roof_type else 'pitched'
        else:
            return 'another dwelling above'
    except Exception as e:
        print(f"Error in determine_roof_type_from_location: {e}")
        return 'another dwelling above'

def get_energy_prediction(total_floor_area, estimated_floor_count, epc_score,
                         wall_insulation, roof_type, roof_insulation, glazing_type,
                         built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type):
    """Energy prediction function with comprehensive error handling"""
    try:
        # Input validation and conversion
        floor_area = safe_float_conversion(total_floor_area, 60.0)
        floor_count = safe_float_conversion(estimated_floor_count, 2.0)
        epc = safe_float_conversion(epc_score, 50.0)
        
        # Validate ranges
        floor_area = max(10.0, min(1000.0, floor_area))
        floor_count = max(1.0, min(10.0, floor_count))
        epc = max(1.0, min(100.0, epc))
        
        # Ensure all string inputs are valid
        wall_insulation = str(wall_insulation) if wall_insulation else 'uninsulated'
        roof_type = str(roof_type) if roof_type else 'pitched'
        roof_insulation = str(roof_insulation) if roof_insulation else 'uninsulated'
        glazing_type = str(glazing_type) if glazing_type else 'single/partial'
        built_form = str(built_form) if built_form else 'mid-terrace'
        main_heat_type = str(main_heat_type) if main_heat_type else 'boiler'
        main_fuel_type = str(main_fuel_type) if main_fuel_type else 'mains gas'
        lookup_age_band = str(lookup_age_band) if lookup_age_band else '1950-1966'
        wall_type = str(wall_type) if wall_type else 'solid'
        
        # U-value calculations with error handling
        U_wall = 0.37 if wall_insulation == 'insulated' else 1.7
        
        if roof_insulation == 'insulated':
            U_roof = 0.25
        elif roof_type == 'flat':
            U_roof = 0.28
        else:
            U_roof = 2.3
        
        U_floor = 0.25
        
        if glazing_type == 'double/triple':
            U_glazing = 2.4
        elif glazing_type == 'secondary':
            U_glazing = 2.82
        else:
            U_glazing = 5.75
        
        # Area calculations
        if roof_type == 'pitched':
            roof_area = floor_area / math.cos(math.radians(30))
        elif roof_type == 'flat':
            roof_area = floor_area
        else:
            roof_area = 0
        
        wall_area = floor_area * 2.1
        floor_area_calc = floor_area
        glazing_area = floor_area * 0.18
        delta_T = 18

        Q_total = (U_wall * wall_area + U_roof * roof_area + U_floor * floor_area_calc + U_glazing * glazing_area) * delta_T

        # Create input data with safe conversions
        rowdict = {
            'epc_score': epc,
            'estimated_floor_count': floor_count,
            'wall_area': wall_area,
            'roof_area': roof_area,
            'floor_area': floor_area_calc,
            'glazing_area': glazing_area,
            'u_value_wall': U_wall,
            'u_value_roof': U_roof,
            'u_value_floor': U_floor,
            'u_value_glazing': U_glazing,
            'Q_total': Q_total,
            'wall_type': wall_type,
            'wall_insulation': wall_insulation,
            'roof_type': roof_type,
            'roof_insulation': roof_insulation,
            'glazing_type': glazing_type,
            'built_form': built_form,
            'main_heat_type': main_heat_type,
            'main_fuel_type': main_fuel_type,
            'lookup_age_band': lookup_age_band
        }
        
        # Create DataFrame and make prediction
        df_input = pd.DataFrame([rowdict])
        df_input = pd.get_dummies(df_input)
        
        # Ensure all required feature columns exist
        for col in feature_cols:
            if col not in df_input.columns:
                df_input[col] = 0
        
        # Select features in correct order
        df_input = df_input[feature_cols]
        
        # Make prediction
        pred = model.predict(df_input)[0]
        
        # Validate prediction result
        if pred < 0:
            print(f"Warning: Negative prediction {pred}, setting to 0")
            pred = 0
        elif pred > 100000:
            print(f"Warning: Very high prediction {pred}")
        
        return float(pred), float(Q_total)
    
    except Exception as e:
        print(f"Error in get_energy_prediction: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        # Return reasonable default values
        default_area = safe_float_conversion(total_floor_area, 60.0)
        default_consumption = default_area * 150  # 150 kWh/m² typical
        default_q_total = default_area * 50      # 50 W/m² typical
        return float(default_consumption), float(default_q_total)

def update_roof_type_visibility(floor_location):
    """Update roof type options visibility based on floor location"""
    try:
        if floor_location == 'top floor':
            return gr.Radio(
                choices=['pitched', 'flat', 'room in roof'],
                value='pitched',
                visible=True
            )
        else:
            return gr.Radio(
                choices=['another dwelling above'],
                value='another dwelling above',
                visible=False
            )
    except Exception as e:
        print(f"Error in update_roof_type_visibility: {e}")
        return gr.Radio(
            choices=['pitched'],
            value='pitched',
            visible=True
        )

def update_roof_insulation_visibility(floor_location):
    """Update roof insulation options visibility based on floor location"""
    try:
        if floor_location == 'top floor':
            return gr.Radio(
                choices=['insulated', 'uninsulated'],
                value='uninsulated',
                visible=True
            )
        else:
            return gr.Radio(
                choices=['another dwelling above'],
                value='another dwelling above',
                visible=False
            )
    except Exception as e:
        print(f"Error in update_roof_insulation_visibility: {e}")
        return gr.Radio(
            choices=['uninsulated'],
            value='uninsulated',
            visible=True
        )

def predict_current_energy_and_show_options(
    lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
    floor_location, roof_type, wall_type, wall_insulation, roof_insulation, 
    glazing_type, main_heat_type, main_fuel_type
):
    """Predict current energy consumption and show available renovation options"""
    try:
        print(f"Starting analysis with inputs: floor_area={total_floor_area}, epc={epc_score}")
        
        # Input validation - ensure all required parameters exist
        required_params = [
            ('lookup_age_band', lookup_age_band),
            ('built_form', built_form),
            ('wall_type', wall_type),
            ('wall_insulation', wall_insulation),
            ('glazing_type', glazing_type),
            ('main_heat_type', main_heat_type),
            ('main_fuel_type', main_fuel_type),
            ('floor_location', floor_location)
        ]
        
        for param_name, param_value in required_params:
            if not param_value:
                raise ValueError(f"Missing required parameter: {param_name}")
        
        # Determine final roof parameters
        final_roof_type = determine_roof_type_from_location(floor_location, roof_type)
        final_roof_insulation = roof_insulation if floor_location == 'top floor' else 'another dwelling above'
        
        print(f"Roof type determined: {final_roof_type}, insulation: {final_roof_insulation}")
        
        # Get energy prediction
        consumption, q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            wall_insulation, final_roof_type, final_roof_insulation, glazing_type,
            built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Energy prediction completed: {consumption} kWh, {q_total} W")
        
        # Calculate costs
        energy_cost = get_energy_cost_per_kwh(main_fuel_type)
        annual_cost = consumption * energy_cost
        
        # Calculate renovation areas
        wall_area, glazing_area, roof_area = calculate_actual_areas(total_floor_area, 'flat')
        
        print(f"Areas calculated: wall={wall_area:.1f}, glazing={glazing_area:.1f}, roof={roof_area:.1f}")
        
        # Performance result display
        performance_result = f"""
## 🏠 Current Building Performance

**📊 Energy Analysis:**
- **Annual Energy Consumption**: {consumption:,.0f} kWh
- **Annual Energy Bills**: £{annual_cost:,.0f} (at £{energy_cost:.3f}/kWh)
- **Total Heating Demand**: {q_total:,.1f} W

**🏢 Building Details:**
- Floor Area: {safe_float_conversion(total_floor_area):.0f} m²
- Floors: {safe_int_conversion(estimated_floor_count)}
- EPC Score: {safe_float_conversion(epc_score):.0f}
- Built Form: {built_form}
- Age Band: {lookup_age_band}
- Floor Location: {floor_location}

**📏 Renovation Areas (for your unit only):**
- External Wall Area: {wall_area:.1f} m² (only exterior walls)
- Window/Glazing Area: {glazing_area:.1f} m² (20% of wall area)
- Roof Area: {roof_area:.1f} m² (if top floor)

**🔧 Current Systems:**
- Wall Type: {wall_type} ({wall_insulation})
- Roof Type: {final_roof_type} ({final_roof_insulation})
- Glazing: {glazing_type}
- Heating: {main_heat_type} ({main_fuel_type})
        """
        
        # Determine available renovation options
        wall_options = ['no_change']
        if wall_insulation == 'uninsulated':
            if wall_type == 'solid':
                wall_options.extend(['internal', 'external'])
            elif wall_type == 'cavity':
                wall_options.extend(['cavity_fill', 'internal', 'external'])
        
        roof_options = ['no_change']
        if floor_location == 'top floor' and final_roof_insulation == 'uninsulated':
            if final_roof_type == 'pitched':
                roof_options.append('loft')
            elif final_roof_type == 'flat':
                roof_options.append('flat_roof')
        
        glazing_options = ['no_change']
        if glazing_type == 'single/partial':
            glazing_options.extend(['double_glazing', 'triple_glazing', 'secondary_glazing'])
        elif glazing_type == 'secondary':
            glazing_options.extend(['double_glazing', 'triple_glazing'])
        
        heating_options = ['no_change']
        if main_heat_type != 'heat pump':
            heating_options.extend(['air_source_heat_pump', 'ground_source_heat_pump'])
        if main_heat_type == 'boiler' and lookup_age_band in ['pre-1920', '1930-1949', '1950-1966', '1967-1982']:
            heating_options.append('gas_boiler_upgrade')
        if main_heat_type != 'boiler' or main_fuel_type != 'electricity':
            heating_options.append('electric_boiler')
        
        fuel_options = ['no_change']
        if main_fuel_type != 'mains gas':
            fuel_options.append('mains gas')
        if main_fuel_type != 'electricity':
            fuel_options.append('electricity')
        
        # Check if any renovation options are available
        show_options = any([
            len(wall_options) > 1,
            len(roof_options) > 1,
            len(glazing_options) > 1,
            len(heating_options) > 1,
            len(fuel_options) > 1
        ])
        
        print(f"Options available: wall={len(wall_options)}, roof={len(roof_options)}, glazing={len(glazing_options)}, heating={len(heating_options)}, fuel={len(fuel_options)}")
        
        return (
            performance_result,
            gr.Radio(choices=wall_options, value='no_change', visible=(len(wall_options) > 1)),
            gr.Radio(choices=roof_options, value='no_change', visible=(len(roof_options) > 1)),
            gr.Radio(choices=glazing_options, value='no_change', visible=(len(glazing_options) > 1)),
            gr.Radio(choices=heating_options, value='no_change', visible=(len(heating_options) > 1)),
            gr.Radio(choices=fuel_options, value='no_change', visible=(len(fuel_options) > 1)),
            gr.Markdown("## 🔧 Available Renovation Options", visible=show_options),
            gr.Button("💰 Calculate Renovation Analysis", visible=show_options)
        )
    
    except Exception as e:
        error_msg = f"❌ Error in building analysis: {str(e)}\n\nPlease check all building parameters and try again."
        print(f"Error in predict_current_energy_and_show_options: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        
        return (
            error_msg,
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Markdown(visible=False),
            gr.Button(visible=False)
        )

def calculate_renovation_analysis(
    lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
    floor_location, roof_type, wall_type, wall_insulation, roof_insulation, 
    glazing_type, main_heat_type, main_fuel_type,
    wall_renovation, roof_renovation, glazing_renovation, heating_renovation, fuel_change
):
    """Calculate renovation costs and energy savings"""
    try:
        print(f"Starting renovation analysis...")
        
        # Handle None values from hidden radio buttons
        wall_renovation = wall_renovation or 'no_change'
        roof_renovation = roof_renovation or 'no_change'
        glazing_renovation = glazing_renovation or 'no_change'
        heating_renovation = heating_renovation or 'no_change'
        fuel_change = fuel_change or 'no_change'
        
        print(f"Renovations selected: wall={wall_renovation}, roof={roof_renovation}, glazing={glazing_renovation}, heating={heating_renovation}, fuel={fuel_change}")
        
        # Determine final roof parameters
        final_roof_type = determine_roof_type_from_location(floor_location, roof_type)
        final_roof_insulation = roof_insulation if floor_location == 'top floor' else 'another dwelling above'
        
        # Check if any renovations are selected
        no_renovations = all([
            wall_renovation == 'no_change',
            roof_renovation == 'no_change',
            glazing_renovation == 'no_change',
            heating_renovation == 'no_change',
            fuel_change == 'no_change'
        ])
        
        if no_renovations:
            return "⚠️ Please select at least one renovation option to see the analysis."
        
        # Calculate renovation areas
        wall_area, glazing_area, roof_area = calculate_actual_areas(total_floor_area, 'flat')
        
        # Get original performance
        original_consumption, original_q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            wall_insulation, final_roof_type, final_roof_insulation, glazing_type,
            built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Original consumption: {original_consumption} kWh")
        
        # Apply renovations
        new_wall_insulation = 'insulated' if wall_renovation != 'no_change' else wall_insulation
        new_roof_insulation = 'insulated' if roof_renovation != 'no_change' else final_roof_insulation
        
        # Glazing upgrades
        new_glazing_type = glazing_type
        if glazing_renovation in ['double_glazing', 'triple_glazing']:
            new_glazing_type = 'double/triple'
        elif glazing_renovation == 'secondary_glazing':
            new_glazing_type = 'secondary'
        
        # Heating system upgrades
        new_heat_type = main_heat_type
        new_fuel_type = main_fuel_type
        
        if heating_renovation in ['air_source_heat_pump', 'ground_source_heat_pump']:
            new_heat_type = 'heat pump'
            new_fuel_type = 'electricity'
        elif heating_renovation == 'gas_boiler_upgrade':
            new_heat_type = 'boiler'
            new_fuel_type = 'mains gas'
        elif heating_renovation == 'electric_boiler':
            new_heat_type = 'boiler'
            new_fuel_type = 'electricity'
        
        # Fuel change override
        if fuel_change != 'no_change':
            new_fuel_type = fuel_change
        
        # Get renovated performance
        renovated_consumption, renovated_q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            new_wall_insulation, final_roof_type, new_roof_insulation, new_glazing_type,
            built_form, new_heat_type, new_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Renovated consumption: {renovated_consumption} kWh")
        
        # Calculate costs
        total_cost = 0
        cost_breakdown = {}
        
        # Wall insulation cost
        if wall_renovation != 'no_change' and wall_renovation in RENOVATION_COSTS['wall_insulation']:
            cost = wall_area * RENOVATION_COSTS['wall_insulation'][wall_renovation]
            total_cost += cost
            cost_breakdown[f'Wall Insulation ({wall_renovation})'] = cost
        
        # Roof insulation cost
        if (roof_renovation != 'no_change' and 
            floor_location == 'top floor' and 
            roof_renovation in RENOVATION_COSTS['roof_insulation']):
            cost = roof_area * RENOVATION_COSTS['roof_insulation'][roof_renovation]
            total_cost += cost
            cost_breakdown[f'Roof Insulation ({roof_renovation})'] = cost
        
        # Glazing cost
        if glazing_renovation != 'no_change' and glazing_renovation in RENOVATION_COSTS['glazing']:
            cost = glazing_area * RENOVATION_COSTS['glazing'][glazing_renovation]
            total_cost += cost
            cost_breakdown[f'Glazing ({glazing_renovation})'] = cost
        
        # Heating system cost
        if heating_renovation != 'no_change' and heating_renovation in RENOVATION_COSTS['heating_system']:
            cost = RENOVATION_COSTS['heating_system'][heating_renovation]
            total_cost += cost
            cost_breakdown[f'Heating ({heating_renovation})'] = cost
        
        # Calculate savings
        annual_savings = max(0, original_consumption - renovated_consumption)
        original_energy_cost = get_energy_cost_per_kwh(main_fuel_type)
        new_energy_cost = get_energy_cost_per_kwh(new_fuel_type)
        
        original_annual_cost = original_consumption * original_energy_cost
        new_annual_cost = renovated_consumption * new_energy_cost
        annual_cost_savings = original_annual_cost - new_annual_cost
        
        # Calculate payback period
        if annual_cost_savings > 0:
            payback_years = total_cost / annual_cost_savings
        else:
            payback_years = float('inf')
        
        # Calculate efficiency improvement
        if original_consumption > 0:
            efficiency_improvement = (annual_savings / original_consumption) * 100
        else:
            efficiency_improvement = 0
        
        print(f"Analysis complete: savings={annual_savings} kWh, cost_savings=£{annual_cost_savings:.0f}, payback={payback_years:.1f} years")
        
        # Generate results
        result_text = f"""
# 🏠 Renovation Analysis Results

## 📊 Performance Impact Summary
| **Metric** | **Current** | **After Renovation** | **Improvement** |
|------------|-------------|---------------------|-----------------|
| **Energy Consumption** | {original_consumption:,.0f} kWh | {renovated_consumption:,.0f} kWh | **{annual_savings:,.0f} kWh** ({efficiency_improvement:.1f}% ↓) |
| **Annual Energy Bills** | £{original_annual_cost:,.0f} | £{new_annual_cost:,.0f} | **£{annual_cost_savings:,.0f}** saved/year |
| **Total Heating Demand** | {original_q_total:,.1f} W | {renovated_q_total:,.1f} W | **{original_q_total - renovated_q_total:,.1f} W** ↓ |

## 💰 Investment Analysis
- **Total Renovation Investment**: £{total_cost:,.0f} *(one-time upfront cost)*
- **Annual Bill Savings**: £{annual_cost_savings:,.0f} *(recurring yearly savings)*
- **Simple Payback Period**: {payback_years:.1f} years
- **25-Year Total Savings**: £{annual_cost_savings * 25:,.0f}

## 🔧 Selected Renovations & Costs
"""
        
        if cost_breakdown:
            for item, cost in cost_breakdown.items():
                percentage = (cost / total_cost * 100) if total_cost > 0 else 0
                result_text += f"- **{item}**: £{cost:,.0f} ({percentage:.1f}%)\n"
        else:
            result_text += "- No renovation costs calculated (free fuel switch only)\n"
        
        result_text += f"""

## 📏 Area Calculations Used:
- **External Wall Area**: {wall_area:.1f} m² (only exterior walls of your unit)
- **Window/Glazing Area**: {glazing_area:.1f} m² (20% of external wall area)
- **Roof Area**: {roof_area:.1f} m² (only if top floor)

## 🏛️ Government Support Information:
- **BUS Grant**: {GOVERNMENT_GRANTS['BUS']['description']}
- **ECO4**: Up to £{GOVERNMENT_GRANTS['ECO4']['max_amount']:,} for eligible households
- **Local Grants**: Check your council for additional support
"""
        
        # Show fuel cost impact if fuel type changes
        if new_fuel_type != main_fuel_type:
            result_text += f"""
## 💡 Fuel Cost Impact:
**Note**: Switching from {main_fuel_type} (£{original_energy_cost:.3f}/kWh) to {new_fuel_type} (£{new_energy_cost:.3f}/kWh)
"""
        
        # Investment recommendation
        if annual_cost_savings > 0:
            if payback_years <= 7:
                result_text += "\n## ✅ **Investment Recommendation: HIGHLY RECOMMENDED**\n**Excellent ROI** - Short payback period with strong long-term savings."
            elif payback_years <= 15:
                result_text += "\n## ⚖️ **Investment Recommendation: RECOMMENDED**\n**Good ROI** - Reasonable payback period with solid benefits."
            else:
                result_text += "\n## ⚠️ **Investment Recommendation: CONSIDER CAREFULLY**\n**Extended Payback** - Prioritize highest-impact measures first."
        else:
            result_text += "\n## ❌ **Investment Recommendation: NOT RECOMMENDED**\n**Negative savings** - This combination increases annual costs. Consider different options."
        
        return result_text
    
    except Exception as e:
        error_msg = f"❌ Error in renovation analysis: {str(e)}\n\nPlease check your selections and try again."
        print(f"Error in calculate_renovation_analysis: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        return error_msg

# Create interface with comprehensive error handling
try:
    with gr.Blocks(theme=gr.themes.Soft(), title="🏠 Smart Energy Renovation Advisor") as demo:
        gr.Markdown("# 🏠 Smart Energy Renovation Advisor")
        gr.Markdown("**Professional energy analysis with verified UK cost data and government grant information**")
        
        with gr.Row():
            # Left column - Building inputs
            with gr.Column(scale=1):
                gr.Markdown("## 🏢 Building Information")
                
                lookup_age_band = gr.Radio(
                    choices=['pre-1920', '1930-1949', '1950-1966', '1967-1982', '1983-1995', '1996-2011', '2012-onwards'],
                    label="📅 Construction Age Band",
                    value="1950-1966"
                )
                
                total_floor_area = gr.Number(
                    label="🏠 Total Floor Area (m²)",
                    value=60,
                    minimum=10,
                    maximum=1000
                )
                
                estimated_floor_count = gr.Number(
                    label="🏢 Above-ground Floors",
                    value=2,
                    minimum=1,
                    maximum=10
                )
                
                epc_score = gr.Number(
                    label="📊 EPC Score",
                    value=50,
                    minimum=1,
                    maximum=100
                )
                
                built_form = gr.Radio(
                    choices=['end-terrace', 'mid-terrace'],
                    label="🏘️ Built Form",
                    value="mid-terrace"
                )
                
                # Floor location and roof type
                floor_location = gr.Radio(
                    choices=['top floor', 'other floor'],
                    label="🏢 Floor Location",
                    value="top floor"
                )
                
                roof_type = gr.Radio(
                    choices=['pitched', 'flat', 'room in roof'],
                    label="🏠 Roof Type",
                    value="pitched"
                )
                
                # Wall information
                with gr.Row():
                    wall_type = gr.Radio(
                        choices=['solid', 'cavity'],
                        label="🧱 Wall Type",
                        value="solid"
                    )
                    wall_insulation = gr.Radio(
                        choices=['insulated', 'uninsulated'],
                        label="🧱 Wall Insulation",
                        value="uninsulated"
                    )
                
                # Roof insulation
                roof_insulation = gr.Radio(
                    choices=['insulated', 'uninsulated'],
                    label="🏠 Roof Insulation",
                    value="uninsulated"
                )
                
                # Other systems
                glazing_type = gr.Radio(
                    choices=['single/partial', 'double/triple', 'secondary'],
                    label="🪟 Glazing Type",
                    value="single/partial"
                )
                
                with gr.Row():
                    main_heat_type = gr.Radio(
                        choices=['boiler', 'communal', 'room/storage heaters', 'heat pump', 'other', 'no heating system'],
                        label="🔥 Main Heating",
                        value="boiler"
                    )
                    main_fuel_type = gr.Radio(
                        choices=['mains gas', 'electricity', 'other', 'no heating system'],
                        label="⚡ Main Fuel",
                        value="mains gas"
                    )
                
                analyze_btn = gr.Button(
                    "📊 Analyze Building Performance",
                    variant="primary",
                    size="lg"
                )
            
            # Right column - Results and options
            with gr.Column(scale=1):
                current_performance = gr.Markdown()
                
                # Renovation options (initially hidden)
                options_title = gr.Markdown("## 🔧 Available Renovation Options", visible=False)
                wall_renovation = gr.Radio(label="🧱 Wall Insulation Upgrade", visible=False)
                roof_renovation = gr.Radio(label="🏠 Roof Insulation Upgrade", visible=False)
                glazing_renovation = gr.Radio(label="🪟 Glazing Upgrade", visible=False)
                heating_renovation = gr.Radio(label="🔥 Heating System Upgrade", visible=False)
                fuel_change = gr.Radio(label="⚡ Fuel Type Change", visible=False)
                
                calculate_btn = gr.Button(
                    "💰 Calculate Renovation Analysis",
                    variant="secondary",
                    size="lg",
                    visible=False
                )
                
                # Results area
                renovation_results = gr.Markdown()
        
        # Event handlers with error handling
        def safe_update_roof_type(floor_location):
            try:
                return update_roof_type_visibility(floor_location)
            except Exception as e:
                print(f"Error updating roof type: {e}")
                return gr.Radio(choices=['pitched'], value='pitched', visible=True)
        
        def safe_update_roof_insulation(floor_location):
            try:
                return update_roof_insulation_visibility(floor_location)
            except Exception as e:
                print(f"Error updating roof insulation: {e}")
                return gr.Radio(choices=['uninsulated'], value='uninsulated', visible=True)
        
        # Connect event handlers
        floor_location.change(
            fn=safe_update_roof_type,
            inputs=[floor_location],
            outputs=[roof_type]
        )
        
        floor_location.change(
            fn=safe_update_roof_insulation,
            inputs=[floor_location],
            outputs=[roof_insulation]
        )
        
        analyze_btn.click(
            fn=predict_current_energy_and_show_options,
            inputs=[lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
                    floor_location, roof_type, wall_type, wall_insulation, roof_insulation,
                    glazing_type, main_heat_type, main_fuel_type],
            outputs=[current_performance, wall_renovation, roof_renovation, glazing_renovation,
                    heating_renovation, fuel_change, options_title, calculate_btn]
        )
        
        calculate_btn.click(
            fn=calculate_renovation_analysis,
            inputs=[lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
                    floor_location, roof_type, wall_type, wall_insulation, roof_insulation,
                    glazing_type, main_heat_type, main_fuel_type,
                    wall_renovation, roof_renovation, glazing_renovation, heating_renovation, fuel_change],
            outputs=[renovation_results]
        )

    # Launch application with proper conditions
    if __name__ == "__main__":
        print("🚀 Launching Smart Energy Renovation Advisor...")
        demo.launch(share=True, debug=False)  # Set debug=False for production
    else:
        print("✅ Application ready for launch")

except Exception as e:
    print(f"❌ Critical error in interface setup: {e}")
    print(f"Traceback: {traceback.format_exc()}")
    raise

d:\Python\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✅ Model files loaded successfully
Model type: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Feature columns count: 26
🚀 Launching Smart Energy Renovation Advisor...
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://47eec421c6af5d53a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Starting analysis with inputs: floor_area=60, epc=50
Roof type determined: pitched, insulation: uninsulated
Energy prediction completed: 21134.262115887217 kWh, 8111.6761373340605 W
Areas calculated: wall=19.4, glazing=3.9, roof=60.0
Options available: wall=3, roof=2, glazing=4, heating=5, fuel=3
Starting renovation analysis...
Renovations selected: wall=no_change, roof=no_change, glazing=no_change, heating=electric_boiler, fuel=no_change
Original consumption: 21134.262115887217 kWh
Renovated consumption: 14018.375515097929 kWh
Analysis complete: savings=7115.886600789288 kWh, cost_savings=£-191, payback=inf years
Starting analysis with inputs: floor_area=60, epc=50
Roof type determined: pitched, insulation: uninsulated
Energy prediction completed: 17457.51824094143 kWh, 8111.6761373340605 W
Areas calculated: wall=19.4, glazing=3.9, roof=60.0
Options available: wall=3, roof=2, glazing=4, heating=5, fuel=2
Starting renovation analysis...
Renovations selected: wall=internal, roof=no_chan

In [10]:
import gradio as gr
import pandas as pd
import numpy as np
import math
import joblib
import joblib
import warnings
import pickle
from sklearn import __version__ as sklearn_version

def load_model_with_compatibility(model_path, feature_path):
    """尝试加载模型，处理版本兼容性问题"""
    try:
        # 尝试正常加载
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            model = joblib.load(model_path)
            feature_cols = joblib.load(feature_path)
        return model, feature_cols, "GB"
    
    except AttributeError as e:
        if "__pyx_unpickle" in str(e):
            print(f"⚠️ GB model incompatible with current scikit-learn {sklearn_version}")
            print("Falling back to RF model...")
            
            # 尝试加载 RF 模型作为备选
            try:
                model = joblib.load('rf_model.joblib')
                feature_cols = joblib.load('rf_features.joblib')
                return model, feature_cols, "RF"
            except Exception as rf_error:
                print(f"❌ RF model also failed: {rf_error}")
                raise
        else:
            raise

# 使用兼容性加载
try:
    model, feature_cols, model_type = load_model_with_compatibility('gb_model.joblib', 'gb_features.joblib')
    print(f"✅ {model_type} Model loaded successfully")
    print(f"Model type: {type(model)}")
    print(f"Feature columns count: {len(feature_cols)}")
except Exception as e:
    print(f"❌ Error loading any model: {e}")
    raise

# Updated renovation costs with realistic pricing (standard costs)
RENOVATION_COSTS = {
    'wall_insulation': {
        'internal': 65,     # £55-75/m²
        'external': 120,    # £100-140/m²
        'cavity_fill': 22   # £18-26/m² - Cavity wall insulation fill
    },
    'roof_insulation': {
        'loft': 18,         # £15-22/m²
        'flat_roof': 80     # £70-90/m²
    },
    'glazing': {
        'double_glazing': 320,      # £280-360/m²
        'triple_glazing': 450,      # £400-500/m²
        'secondary_glazing': 150    # £130-170/m²
    },
    'heating_system': {
        # These costs are AFTER BUS grant (already deducted)
        'air_source_heat_pump': 6500,     # £14,000 - £7,500 BUS grant
        'ground_source_heat_pump': 12000, # £18,000 - £6,000 BUS grant
        'gas_boiler_upgrade': 2400,       # Standard cost
        'electric_boiler': 1800           # Standard cost
    }
}

# Energy costs from Ofgem Price Cap (October 2024)
ENERGY_COSTS = {
    'electricity': 0.285,  # £/kWh 
    'gas': 0.073,         # £/kWh
    'mixed': 0.18         # Average for mixed systems
}

# Government grants information
GOVERNMENT_GRANTS = {
    'BUS': {
        'air_source_heat_pump': 7500,
        'ground_source_heat_pump': 6000,
        'description': 'Boiler Upgrade Scheme (BUS) grants already deducted from heat pump costs above'
    },
    'ECO4': {
        'max_amount': 10000,
        'description': 'Energy Company Obligation - for eligible low-income households'
    }
}

def safe_float_conversion(value, default=0.0):
    """Safely convert value to float"""
    try:
        if value is None:
            return default
        return float(value)
    except (ValueError, TypeError):
        print(f"Warning: Could not convert {value} to float, using default {default}")
        return default

def safe_int_conversion(value, default=1):
    """Safely convert value to int"""
    try:
        if value is None:
            return default
        return int(float(value))  # Convert through float first to handle "2.0" strings
    except (ValueError, TypeError):
        print(f"Warning: Could not convert {value} to int, using default {default}")
        return default

def get_energy_cost_per_kwh(fuel_type):
    """Get energy cost per kWh with comprehensive error handling"""
    try:
        if fuel_type is None:
            return ENERGY_COSTS['mixed']
        
        fuel_type_str = str(fuel_type).lower().strip()
        
        if 'electric' in fuel_type_str:
            return ENERGY_COSTS['electricity']
        elif 'gas' in fuel_type_str:
            return ENERGY_COSTS['gas']
        else:
            return ENERGY_COSTS['mixed']
    except Exception as e:
        print(f"Warning: Error getting energy cost for {fuel_type}: {e}")
        return ENERGY_COSTS['mixed']

def calculate_actual_areas(total_floor_area, building_type='flat'):
    """Calculate actual renovation areas with comprehensive validation"""
    try:
        # Input validation and conversion
        floor_area = safe_float_conversion(total_floor_area, 60.0)
        
        if floor_area <= 0:
            print(f"Warning: Invalid floor area {floor_area}, using default 60")
            floor_area = 60.0
        
        if floor_area > 2000:  # Sanity check
            print(f"Warning: Very large floor area: {floor_area} m²")

        if building_type == 'flat':
            # Conservative calculation for flats
            wall_perimeter = math.sqrt(floor_area) * 2.5
            wall_area = wall_perimeter * 2.5 * 0.4  # Only 40% is external wall
            glazing_area = wall_area * 0.2  # 20% of wall area
            roof_area = floor_area
        else:  # house
            wall_area = floor_area * 1.8
            glazing_area = floor_area * 0.15
            roof_area = floor_area / math.cos(math.radians(30))
        
        # Ensure all areas are positive
        wall_area = max(wall_area, 1.0)
        glazing_area = max(glazing_area, 1.0)
        roof_area = max(roof_area, 1.0)
        
        return wall_area, glazing_area, roof_area
    
    except Exception as e:
        print(f"Error in calculate_actual_areas: {e}")
        # Return safe default values
        default_area = safe_float_conversion(total_floor_area, 60.0)
        return default_area * 0.3, default_area * 0.06, default_area

def determine_roof_type_from_location(floor_location, roof_type):
    """Determine final roof type based on floor location"""
    try:
        if floor_location == 'top floor':
            return str(roof_type) if roof_type else 'pitched'
        else:
            return 'another dwelling above'
    except Exception as e:
        print(f"Error in determine_roof_type_from_location: {e}")
        return 'another dwelling above'

def get_energy_prediction(total_floor_area, estimated_floor_count, epc_score,
                         wall_insulation, roof_type, roof_insulation, glazing_type,
                         built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type):
    """Energy prediction function with comprehensive error handling"""
    try:
        # Input validation and conversion
        floor_area = safe_float_conversion(total_floor_area, 60.0)
        floor_count = safe_float_conversion(estimated_floor_count, 2.0)
        epc = safe_float_conversion(epc_score, 50.0)
        
        # Validate ranges
        floor_area = max(10.0, min(1000.0, floor_area))
        floor_count = max(1.0, min(10.0, floor_count))
        epc = max(1.0, min(100.0, epc))
        
        # Ensure all string inputs are valid
        wall_insulation = str(wall_insulation) if wall_insulation else 'uninsulated'
        roof_type = str(roof_type) if roof_type else 'pitched'
        roof_insulation = str(roof_insulation) if roof_insulation else 'uninsulated'
        glazing_type = str(glazing_type) if glazing_type else 'single/partial'
        built_form = str(built_form) if built_form else 'mid-terrace'
        main_heat_type = str(main_heat_type) if main_heat_type else 'boiler'
        main_fuel_type = str(main_fuel_type) if main_fuel_type else 'mains gas'
        lookup_age_band = str(lookup_age_band) if lookup_age_band else '1950-1966'
        wall_type = str(wall_type) if wall_type else 'solid'
        
        # U-value calculations with error handling
        U_wall = 0.37 if wall_insulation == 'insulated' else 1.7
        
        if roof_insulation == 'insulated':
            U_roof = 0.25
        elif roof_type == 'flat':
            U_roof = 0.28
        else:
            U_roof = 2.3
        
        U_floor = 0.25
        
        if glazing_type == 'double/triple':
            U_glazing = 2.4
        elif glazing_type == 'secondary':
            U_glazing = 2.82
        else:
            U_glazing = 5.75
        
        # Area calculations
        if roof_type == 'pitched':
            roof_area = floor_area / math.cos(math.radians(30))
        elif roof_type == 'flat':
            roof_area = floor_area
        else:
            roof_area = 0
        
        wall_area = floor_area * 2.1
        floor_area_calc = floor_area
        glazing_area = floor_area * 0.18
        delta_T = 18

        Q_total = (U_wall * wall_area + U_roof * roof_area + U_floor * floor_area_calc + U_glazing * glazing_area) * delta_T

        # Create input data with safe conversions
        rowdict = {
            'epc_score': epc,
            'estimated_floor_count': floor_count,
            'wall_area': wall_area,
            'roof_area': roof_area,
            'floor_area': floor_area_calc,
            'glazing_area': glazing_area,
            'u_value_wall': U_wall,
            'u_value_roof': U_roof,
            'u_value_floor': U_floor,
            'u_value_glazing': U_glazing,
            'Q_total': Q_total,
            'wall_type': wall_type,
            'wall_insulation': wall_insulation,
            'roof_type': roof_type,
            'roof_insulation': roof_insulation,
            'glazing_type': glazing_type,
            'built_form': built_form,
            'main_heat_type': main_heat_type,
            'main_fuel_type': main_fuel_type,
            'lookup_age_band': lookup_age_band
        }
        
        # Create DataFrame and make prediction
        df_input = pd.DataFrame([rowdict])
        df_input = pd.get_dummies(df_input)
        
        # Ensure all required feature columns exist
        for col in feature_cols:
            if col not in df_input.columns:
                df_input[col] = 0
        
        # Select features in correct order
        df_input = df_input[feature_cols]
        
        # Make prediction
        pred = model.predict(df_input)[0]
        
        # Validate prediction result
        if pred < 0:
            print(f"Warning: Negative prediction {pred}, setting to 0")
            pred = 0
        elif pred > 100000:
            print(f"Warning: Very high prediction {pred}")
        
        return float(pred), float(Q_total)
    
    except Exception as e:
        print(f"Error in get_energy_prediction: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        # Return reasonable default values
        default_area = safe_float_conversion(total_floor_area, 60.0)
        default_consumption = default_area * 150  # 150 kWh/m² typical
        default_q_total = default_area * 50      # 50 W/m² typical
        return float(default_consumption), float(default_q_total)

def update_roof_type_visibility(floor_location):
    """Update roof type options visibility based on floor location"""
    try:
        if floor_location == 'top floor':
            return gr.Radio(
                choices=['pitched', 'flat', 'room in roof'],
                value='pitched',
                visible=True
            )
        else:
            return gr.Radio(
                choices=['another dwelling above'],
                value='another dwelling above',
                visible=False
            )
    except Exception as e:
        print(f"Error in update_roof_type_visibility: {e}")
        return gr.Radio(
            choices=['pitched'],
            value='pitched',
            visible=True
        )

def update_roof_insulation_visibility(floor_location):
    """Update roof insulation options visibility based on floor location"""
    try:
        if floor_location == 'top floor':
            return gr.Radio(
                choices=['insulated', 'uninsulated'],
                value='uninsulated',
                visible=True
            )
        else:
            return gr.Radio(
                choices=['another dwelling above'],
                value='another dwelling above',
                visible=False
            )
    except Exception as e:
        print(f"Error in update_roof_insulation_visibility: {e}")
        return gr.Radio(
            choices=['uninsulated'],
            value='uninsulated',
            visible=True
        )

def predict_current_energy_and_show_options(
    lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
    floor_location, roof_type, wall_type, wall_insulation, roof_insulation, 
    glazing_type, main_heat_type, main_fuel_type
):
    """Predict current energy consumption and show available renovation options"""
    try:
        print(f"Starting analysis with inputs: floor_area={total_floor_area}, epc={epc_score}")
        
        # Input validation - ensure all required parameters exist
        required_params = [
            ('lookup_age_band', lookup_age_band),
            ('built_form', built_form),
            ('wall_type', wall_type),
            ('wall_insulation', wall_insulation),
            ('glazing_type', glazing_type),
            ('main_heat_type', main_heat_type),
            ('main_fuel_type', main_fuel_type),
            ('floor_location', floor_location)
        ]
        
        for param_name, param_value in required_params:
            if not param_value:
                raise ValueError(f"Missing required parameter: {param_name}")
        
        # Determine final roof parameters
        final_roof_type = determine_roof_type_from_location(floor_location, roof_type)
        final_roof_insulation = roof_insulation if floor_location == 'top floor' else 'another dwelling above'
        
        print(f"Roof type determined: {final_roof_type}, insulation: {final_roof_insulation}")
        
        # Get energy prediction
        consumption, q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            wall_insulation, final_roof_type, final_roof_insulation, glazing_type,
            built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Energy prediction completed: {consumption} kWh, {q_total} W")
        
        # Calculate costs
        energy_cost = get_energy_cost_per_kwh(main_fuel_type)
        annual_cost = consumption * energy_cost
        
        # Calculate renovation areas
        wall_area, glazing_area, roof_area = calculate_actual_areas(total_floor_area, 'flat')
        
        print(f"Areas calculated: wall={wall_area:.1f}, glazing={glazing_area:.1f}, roof={roof_area:.1f}")
        
        # Performance result display
        performance_result = f"""
## 🏠 Current Building Performance

**📊 Energy Analysis:**
- **Annual Energy Consumption**: {consumption:,.0f} kWh
- **Annual Energy Bills**: £{annual_cost:,.0f} (at £{energy_cost:.3f}/kWh)
- **Total Heating Demand**: {q_total:,.1f} W

**🏢 Building Details:**
- Floor Area: {safe_float_conversion(total_floor_area):.0f} m²
- Floors: {safe_int_conversion(estimated_floor_count)}
- EPC Score: {safe_float_conversion(epc_score):.0f}
- Built Form: {built_form}
- Age Band: {lookup_age_band}
- Floor Location: {floor_location}

**📏 Renovation Areas (for your unit only):**
- External Wall Area: {wall_area:.1f} m² (only exterior walls)
- Window/Glazing Area: {glazing_area:.1f} m² (20% of wall area)
- Roof Area: {roof_area:.1f} m² (if top floor)

**🔧 Current Systems:**
- Wall Type: {wall_type} ({wall_insulation})
- Roof Type: {final_roof_type} ({final_roof_insulation})
- Glazing: {glazing_type}
- Heating: {main_heat_type} ({main_fuel_type})
        """
        
        # Determine available renovation options
        wall_options = ['no_change']
        if wall_insulation == 'uninsulated':
            if wall_type == 'solid':
                wall_options.extend(['internal', 'external'])
            elif wall_type == 'cavity':
                wall_options.extend(['cavity_fill', 'internal', 'external'])
        
        roof_options = ['no_change']
        if floor_location == 'top floor' and final_roof_insulation == 'uninsulated':
            if final_roof_type == 'pitched':
                roof_options.append('loft')
            elif final_roof_type == 'flat':
                roof_options.append('flat_roof')
        
        glazing_options = ['no_change']
        if glazing_type == 'single/partial':
            glazing_options.extend(['double_glazing', 'triple_glazing', 'secondary_glazing'])
        elif glazing_type == 'secondary':
            glazing_options.extend(['double_glazing', 'triple_glazing'])
        
        heating_options = ['no_change']
        if main_heat_type != 'heat pump':
            heating_options.extend(['air_source_heat_pump', 'ground_source_heat_pump'])
        if main_heat_type == 'boiler' and lookup_age_band in ['pre-1920', '1930-1949', '1950-1966', '1967-1982']:
            heating_options.append('gas_boiler_upgrade')
        if main_heat_type != 'boiler' or main_fuel_type != 'electricity':
            heating_options.append('electric_boiler')
        
        fuel_options = ['no_change']
        if main_fuel_type != 'mains gas':
            fuel_options.append('mains gas')
        if main_fuel_type != 'electricity':
            fuel_options.append('electricity')
        
        # Check if any renovation options are available
        show_options = any([
            len(wall_options) > 1,
            len(roof_options) > 1,
            len(glazing_options) > 1,
            len(heating_options) > 1,
            len(fuel_options) > 1
        ])
        
        print(f"Options available: wall={len(wall_options)}, roof={len(roof_options)}, glazing={len(glazing_options)}, heating={len(heating_options)}, fuel={len(fuel_options)}")
        
        return (
            performance_result,
            gr.Radio(choices=wall_options, value='no_change', visible=(len(wall_options) > 1)),
            gr.Radio(choices=roof_options, value='no_change', visible=(len(roof_options) > 1)),
            gr.Radio(choices=glazing_options, value='no_change', visible=(len(glazing_options) > 1)),
            gr.Radio(choices=heating_options, value='no_change', visible=(len(heating_options) > 1)),
            gr.Radio(choices=fuel_options, value='no_change', visible=(len(fuel_options) > 1)),
            gr.Markdown("## 🔧 Available Renovation Options", visible=show_options),
            gr.Button("💰 Calculate Renovation Analysis", visible=show_options)
        )
    
    except Exception as e:
        error_msg = f"❌ Error in building analysis: {str(e)}\n\nPlease check all building parameters and try again."
        print(f"Error in predict_current_energy_and_show_options: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        
        return (
            error_msg,
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Radio(choices=['no_change'], value='no_change', visible=False),
            gr.Markdown(visible=False),
            gr.Button(visible=False)
        )

def calculate_renovation_analysis(
    lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
    floor_location, roof_type, wall_type, wall_insulation, roof_insulation, 
    glazing_type, main_heat_type, main_fuel_type,
    wall_renovation, roof_renovation, glazing_renovation, heating_renovation, fuel_change
):
    """Calculate renovation costs and energy savings"""
    try:
        print(f"Starting renovation analysis...")
        
        # Handle None values from hidden radio buttons
        wall_renovation = wall_renovation or 'no_change'
        roof_renovation = roof_renovation or 'no_change'
        glazing_renovation = glazing_renovation or 'no_change'
        heating_renovation = heating_renovation or 'no_change'
        fuel_change = fuel_change or 'no_change'
        
        print(f"Renovations selected: wall={wall_renovation}, roof={roof_renovation}, glazing={glazing_renovation}, heating={heating_renovation}, fuel={fuel_change}")
        
        # Determine final roof parameters
        final_roof_type = determine_roof_type_from_location(floor_location, roof_type)
        final_roof_insulation = roof_insulation if floor_location == 'top floor' else 'another dwelling above'
        
        # Check if any renovations are selected
        no_renovations = all([
            wall_renovation == 'no_change',
            roof_renovation == 'no_change',
            glazing_renovation == 'no_change',
            heating_renovation == 'no_change',
            fuel_change == 'no_change'
        ])
        
        if no_renovations:
            return "⚠️ Please select at least one renovation option to see the analysis."
        
        # Calculate renovation areas
        wall_area, glazing_area, roof_area = calculate_actual_areas(total_floor_area, 'flat')
        
        # Get original performance
        original_consumption, original_q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            wall_insulation, final_roof_type, final_roof_insulation, glazing_type,
            built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Original consumption: {original_consumption} kWh")
        
        # Apply renovations
        new_wall_insulation = 'insulated' if wall_renovation != 'no_change' else wall_insulation
        new_roof_insulation = 'insulated' if roof_renovation != 'no_change' else final_roof_insulation
        
        # Glazing upgrades
        new_glazing_type = glazing_type
        if glazing_renovation in ['double_glazing', 'triple_glazing']:
            new_glazing_type = 'double/triple'
        elif glazing_renovation == 'secondary_glazing':
            new_glazing_type = 'secondary'
        
        # Heating system upgrades
        new_heat_type = main_heat_type
        new_fuel_type = main_fuel_type
        
        if heating_renovation in ['air_source_heat_pump', 'ground_source_heat_pump']:
            new_heat_type = 'heat pump'
            new_fuel_type = 'electricity'
        elif heating_renovation == 'gas_boiler_upgrade':
            new_heat_type = 'boiler'
            new_fuel_type = 'mains gas'
        elif heating_renovation == 'electric_boiler':
            new_heat_type = 'boiler'
            new_fuel_type = 'electricity'
        
        # Fuel change override
        if fuel_change != 'no_change':
            new_fuel_type = fuel_change
        
        # Get renovated performance
        renovated_consumption, renovated_q_total = get_energy_prediction(
            total_floor_area, estimated_floor_count, epc_score,
            new_wall_insulation, final_roof_type, new_roof_insulation, new_glazing_type,
            built_form, new_heat_type, new_fuel_type, lookup_age_band, wall_type
        )
        
        print(f"Renovated consumption: {renovated_consumption} kWh")
        
        # Calculate costs
        total_cost = 0
        cost_breakdown = {}
        
        # Wall insulation cost
        if wall_renovation != 'no_change' and wall_renovation in RENOVATION_COSTS['wall_insulation']:
            cost = wall_area * RENOVATION_COSTS['wall_insulation'][wall_renovation]
            total_cost += cost
            cost_breakdown[f'Wall Insulation ({wall_renovation})'] = cost
        
        # Roof insulation cost
        if (roof_renovation != 'no_change' and 
            floor_location == 'top floor' and 
            roof_renovation in RENOVATION_COSTS['roof_insulation']):
            cost = roof_area * RENOVATION_COSTS['roof_insulation'][roof_renovation]
            total_cost += cost
            cost_breakdown[f'Roof Insulation ({roof_renovation})'] = cost
        
        # Glazing cost
        if glazing_renovation != 'no_change' and glazing_renovation in RENOVATION_COSTS['glazing']:
            cost = glazing_area * RENOVATION_COSTS['glazing'][glazing_renovation]
            total_cost += cost
            cost_breakdown[f'Glazing ({glazing_renovation})'] = cost
        
        # Heating system cost
        if heating_renovation != 'no_change' and heating_renovation in RENOVATION_COSTS['heating_system']:
            cost = RENOVATION_COSTS['heating_system'][heating_renovation]
            total_cost += cost
            cost_breakdown[f'Heating ({heating_renovation})'] = cost
        
        # Calculate savings
        annual_savings = max(0, original_consumption - renovated_consumption)
        original_energy_cost = get_energy_cost_per_kwh(main_fuel_type)
        new_energy_cost = get_energy_cost_per_kwh(new_fuel_type)
        
        original_annual_cost = original_consumption * original_energy_cost
        new_annual_cost = renovated_consumption * new_energy_cost
        annual_cost_savings = original_annual_cost - new_annual_cost
        
        # Calculate payback period
        if annual_cost_savings > 0:
            payback_years = total_cost / annual_cost_savings
        else:
            payback_years = float('inf')
        
        # Calculate efficiency improvement
        if original_consumption > 0:
            efficiency_improvement = (annual_savings / original_consumption) * 100
        else:
            efficiency_improvement = 0
        
        print(f"Analysis complete: savings={annual_savings} kWh, cost_savings=£{annual_cost_savings:.0f}, payback={payback_years:.1f} years")
        
        # Generate results
        result_text = f"""
# 🏠 Renovation Analysis Results

## 📊 Performance Impact Summary
| **Metric** | **Current** | **After Renovation** | **Improvement** |
|------------|-------------|---------------------|-----------------|
| **Energy Consumption** | {original_consumption:,.0f} kWh | {renovated_consumption:,.0f} kWh | **{annual_savings:,.0f} kWh** ({efficiency_improvement:.1f}% ↓) |
| **Annual Energy Bills** | £{original_annual_cost:,.0f} | £{new_annual_cost:,.0f} | **£{annual_cost_savings:,.0f}** saved/year |
| **Total Heating Demand** | {original_q_total:,.1f} W | {renovated_q_total:,.1f} W | **{original_q_total - renovated_q_total:,.1f} W** ↓ |

## 💰 Investment Analysis
- **Total Renovation Investment**: £{total_cost:,.0f} *(one-time upfront cost)*
- **Annual Bill Savings**: £{annual_cost_savings:,.0f} *(recurring yearly savings)*
- **Simple Payback Period**: {payback_years:.1f} years
- **25-Year Total Savings**: £{annual_cost_savings * 25:,.0f}

## 🔧 Selected Renovations & Costs
"""
        
        if cost_breakdown:
            for item, cost in cost_breakdown.items():
                percentage = (cost / total_cost * 100) if total_cost > 0 else 0
                result_text += f"- **{item}**: £{cost:,.0f} ({percentage:.1f}%)\n"
        else:
            result_text += "- No renovation costs calculated (free fuel switch only)\n"
        
        result_text += f"""

## 📏 Area Calculations Used:
- **External Wall Area**: {wall_area:.1f} m² (only exterior walls of your unit)
- **Window/Glazing Area**: {glazing_area:.1f} m² (20% of external wall area)
- **Roof Area**: {roof_area:.1f} m² (only if top floor)

## 🏛️ Government Support Information:
- **BUS Grant**: {GOVERNMENT_GRANTS['BUS']['description']}
- **ECO4**: Up to £{GOVERNMENT_GRANTS['ECO4']['max_amount']:,} for eligible households
- **Local Grants**: Check your council for additional support
"""
        
        # Show fuel cost impact if fuel type changes
        if new_fuel_type != main_fuel_type:
            result_text += f"""
## 💡 Fuel Cost Impact:
**Note**: Switching from {main_fuel_type} (£{original_energy_cost:.3f}/kWh) to {new_fuel_type} (£{new_energy_cost:.3f}/kWh)
"""
        
        # Investment recommendation
        if annual_cost_savings > 0:
            if payback_years <= 7:
                result_text += "\n## ✅ **Investment Recommendation: HIGHLY RECOMMENDED**\n**Excellent ROI** - Short payback period with strong long-term savings."
            elif payback_years <= 15:
                result_text += "\n## ⚖️ **Investment Recommendation: RECOMMENDED**\n**Good ROI** - Reasonable payback period with solid benefits."
            else:
                result_text += "\n## ⚠️ **Investment Recommendation: CONSIDER CAREFULLY**\n**Extended Payback** - Prioritize highest-impact measures first."
        else:
            result_text += "\n## ❌ **Investment Recommendation: NOT RECOMMENDED**\n**Negative savings** - This combination increases annual costs. Consider different options."
        
        return result_text
    
    except Exception as e:
        error_msg = f"❌ Error in renovation analysis: {str(e)}\n\nPlease check your selections and try again."
        print(f"Error in calculate_renovation_analysis: {e}")
        print(f"Traceback: {traceback.format_exc()}")
        return error_msg

# Create interface with comprehensive error handling
try:
    with gr.Blocks(theme=gr.themes.Soft(), title="🏠 Smart Energy Renovation Advisor") as demo:
        gr.Markdown("# 🏠 Smart Energy Renovation Advisor")
        gr.Markdown("**Professional energy analysis with verified UK cost data and government grant information**")
        
        with gr.Row():
            # Left column - Building inputs
            with gr.Column(scale=1):
                gr.Markdown("## 🏢 Building Information")
                
                lookup_age_band = gr.Radio(
                    choices=['pre-1920', '1930-1949', '1950-1966', '1967-1982', '1983-1995', '1996-2011', '2012-onwards'],
                    label="📅 Construction Age Band",
                    value="1950-1966"
                )
                
                total_floor_area = gr.Number(
                    label="🏠 Total Floor Area (m²)",
                    value=60,
                    minimum=10,
                    maximum=1000
                )
                
                estimated_floor_count = gr.Number(
                    label="🏢 Above-ground Floors",
                    value=2,
                    minimum=1,
                    maximum=10
                )
                
                epc_score = gr.Number(
                    label="📊 EPC Score",
                    value=50,
                    minimum=1,
                    maximum=100
                )
                
                built_form = gr.Radio(
                    choices=['end-terrace', 'mid-terrace'],
                    label="🏘️ Built Form",
                    value="mid-terrace"
                )
                
                # Floor location and roof type
                floor_location = gr.Radio(
                    choices=['top floor', 'other floor'],
                    label="🏢 Floor Location",
                    value="top floor"
                )
                
                roof_type = gr.Radio(
                    choices=['pitched', 'flat', 'room in roof'],
                    label="🏠 Roof Type",
                    value="pitched"
                )
                
                # Wall information
                with gr.Row():
                    wall_type = gr.Radio(
                        choices=['solid', 'cavity'],
                        label="🧱 Wall Type",
                        value="solid"
                    )
                    wall_insulation = gr.Radio(
                        choices=['insulated', 'uninsulated'],
                        label="🧱 Wall Insulation",
                        value="uninsulated"
                    )
                
                # Roof insulation
                roof_insulation = gr.Radio(
                    choices=['insulated', 'uninsulated'],
                    label="🏠 Roof Insulation",
                    value="uninsulated"
                )
                
                # Other systems
                glazing_type = gr.Radio(
                    choices=['single/partial', 'double/triple', 'secondary'],
                    label="🪟 Glazing Type",
                    value="single/partial"
                )
                
                with gr.Row():
                    main_heat_type = gr.Radio(
                        choices=['boiler', 'communal', 'room/storage heaters', 'heat pump', 'other', 'no heating system'],
                        label="🔥 Main Heating",
                        value="boiler"
                    )
                    main_fuel_type = gr.Radio(
                        choices=['mains gas', 'electricity', 'other', 'no heating system'],
                        label="⚡ Main Fuel",
                        value="mains gas"
                    )
                
                analyze_btn = gr.Button(
                    "📊 Analyze Building Performance",
                    variant="primary",
                    size="lg"
                )
            
            # Right column - Results and options
            with gr.Column(scale=1):
                current_performance = gr.Markdown()
                
                # Renovation options (initially hidden)
                options_title = gr.Markdown("## 🔧 Available Renovation Options", visible=False)
                wall_renovation = gr.Radio(label="🧱 Wall Insulation Upgrade", visible=False)
                roof_renovation = gr.Radio(label="🏠 Roof Insulation Upgrade", visible=False)
                glazing_renovation = gr.Radio(label="🪟 Glazing Upgrade", visible=False)
                heating_renovation = gr.Radio(label="🔥 Heating System Upgrade", visible=False)
                fuel_change = gr.Radio(label="⚡ Fuel Type Change", visible=False)
                
                calculate_btn = gr.Button(
                    "💰 Calculate Renovation Analysis",
                    variant="secondary",
                    size="lg",
                    visible=False
                )
                
                # Results area
                renovation_results = gr.Markdown()
        
        # Event handlers with error handling
        def safe_update_roof_type(floor_location):
            try:
                return update_roof_type_visibility(floor_location)
            except Exception as e:
                print(f"Error updating roof type: {e}")
                return gr.Radio(choices=['pitched'], value='pitched', visible=True)
        
        def safe_update_roof_insulation(floor_location):
            try:
                return update_roof_insulation_visibility(floor_location)
            except Exception as e:
                print(f"Error updating roof insulation: {e}")
                return gr.Radio(choices=['uninsulated'], value='uninsulated', visible=True)
        
        # Connect event handlers
        floor_location.change(
            fn=safe_update_roof_type,
            inputs=[floor_location],
            outputs=[roof_type]
        )
        
        floor_location.change(
            fn=safe_update_roof_insulation,
            inputs=[floor_location],
            outputs=[roof_insulation]
        )
        
        analyze_btn.click(
            fn=predict_current_energy_and_show_options,
            inputs=[lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
                    floor_location, roof_type, wall_type, wall_insulation, roof_insulation,
                    glazing_type, main_heat_type, main_fuel_type],
            outputs=[current_performance, wall_renovation, roof_renovation, glazing_renovation,
                    heating_renovation, fuel_change, options_title, calculate_btn]
        )
        
        calculate_btn.click(
            fn=calculate_renovation_analysis,
            inputs=[lookup_age_band, total_floor_area, estimated_floor_count, epc_score, built_form,
                    floor_location, roof_type, wall_type, wall_insulation, roof_insulation,
                    glazing_type, main_heat_type, main_fuel_type,
                    wall_renovation, roof_renovation, glazing_renovation, heating_renovation, fuel_change],
            outputs=[renovation_results]
        )

    # Launch application with proper conditions
    if __name__ == "__main__":
        print("🚀 Launching Smart Energy Renovation Advisor...")
        demo.launch(share=True, debug=False)  # Set debug=False for production
    else:
        print("✅ Application ready for launch")

except Exception as e:
    print(f"❌ Critical error in interface setup: {e}")
    print(f"Traceback: {traceback.format_exc()}")
    raise

d:\Python\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


⚠️ GB model incompatible with current scikit-learn 1.7.0
Falling back to RF model...
✅ RF Model loaded successfully
Model type: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Feature columns count: 26
🚀 Launching Smart Energy Renovation Advisor...
* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://598b98baebc8f054a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Starting analysis with inputs: floor_area=60, epc=50
Roof type determined: pitched, insulation: uninsulated
Energy prediction completed: 17457.51824094143 kWh, 8111.6761373340605 W
Areas calculated: wall=19.4, glazing=3.9, roof=60.0
Options available: wall=3, roof=2, glazing=4, heating=5, fuel=2
Starting analysis with inputs: floor_area=60, epc=50
Roof type determined: pitched, insulation: uninsulated
Energy prediction completed: 17457.51824094143 kWh, 8111.6761373340605 W
Areas calculated: wall=19.4, glazing=3.9, roof=60.0
Options available: wall=4, roof=2, glazing=4, heating=5, fuel=2
Starting renovation analysis...
Renovations selected: wall=cavity_fill, roof=no_change, glazing=no_change, heating=no_change, fuel=no_change
Original consumption: 17457.51824094143 kWh
Renovated consumption: 16181.235812770816 kWh
Analysis complete: savings=1276.2824281706144 kWh, cost_savings=£93, payback=4.6 years


In [7]:
pip install scikit-learn==1.7.0  # 降级到旧版本

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
